In [2]:
import os
import json
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder






In [3]:
# === 1. Device configuration ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")





In [4]:
# === 2 Define ImageNet preprocessing ===
mean_norms = np.array([0.485, 0.456, 0.406])
std_norms = np.array([0.229, 0.224, 0.225])

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_norms, std=std_norms)
])


In [5]:
# === 3。 Function to reload dataset with corrected label mapping ===
def reload_testset_after_fix(dataset_path, transform, fixed_class_to_idx):
    """
    Reload the ImageFolder dataset and replace the class_to_idx and samples
    to reflect correct ImageNet indices based on synset IDs.
    """
    # Reload the dataset
    testset = ImageFolder(root=dataset_path, transform=transform)

    # Replace class_to_idx with the fixed mapping
    testset.class_to_idx = fixed_class_to_idx

    # Manually reassign each sample’s label using the new mapping
    new_samples = []
    for path, _ in testset.samples:
        class_name = os.path.basename(os.path.dirname(path))
        if class_name in fixed_class_to_idx:
            new_samples.append((path, fixed_class_to_idx[class_name]))
        else:
            print(f"⚠️ Skipping unmatched class: {class_name}")
    testset.samples = new_samples

    testloader = DataLoader(testset, batch_size=16, shuffle=False)

    print("✅ Fully refreshed dataset with correct label indices:")
    for i in range(3):
        _, label = testset[i]
        print(f"Image {i} label index: {label}")

    return testset, testloader


In [6]:
# === 4. Build the corrected synset → index mapping ===
dataset_path = r"C:\Users\AthrunZala\Downloads\TestDataSet"
filtered_path = os.path.join(dataset_path, "filtered_classes")

# Load dataset to get original class names
testset_raw = ImageFolder(root=filtered_path, transform=transform)

# Load official ImageNet synset-to-index mapping
with open(os.path.join(dataset_path, "imagenet_class_index.json"), "r") as f:
    imagenet_class_index = json.load(f)
synset_to_idx = {v[0]: int(k) for k, v in imagenet_class_index.items()}

# Match synset folders to correct ImageNet index
fixed_class_to_idx = {
    synset: synset_to_idx[synset]
    for synset in testset_raw.class_to_idx
    if synset in synset_to_idx
}




In [7]:
# === 5. Refresh dataset using corrected mapping ===
testset, testloader = reload_testset_after_fix(
    dataset_path=filtered_path,
    transform=transform,
    fixed_class_to_idx=fixed_class_to_idx
)

✅ Fully refreshed dataset with correct label indices:
Image 0 label index: 401
Image 1 label index: 401
Image 2 label index: 401


In [8]:
# === 6. Load pre-trained ResNet-34 ===
model = models.resnet34(weights='IMAGENET1K_V1').to(device).eval()


In [9]:
# === 7. Accuracy calculation ===
def accuracy(output, target, topk=(1, 5)):
    maxk = max(topk)
    _, pred = output.topk(maxk, 1, True, True)
    correct = pred.eq(target.view(-1, 1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = correct[:, :k].sum().item()
        res.append(correct_k)
    return res

In [10]:
def evaluate(model, dataloader):
    model.eval()
    top1_total = 0
    top5_total = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            top1, top5 = accuracy(outputs, labels)
            top1_total += top1
            top5_total += top5
            total += labels.size(0)
    return top1_total / total, top5_total / total



In [11]:
# === 8. FGSM attack ===
def fgsm_attack(model, images, labels, epsilon):
    images = images.clone().detach().requires_grad_()
    outputs = model(images)
    loss = F.cross_entropy(outputs, labels)
    model.zero_grad()
    loss.backward()
    perturbed = images + epsilon * images.grad.sign()
    return torch.clamp(perturbed, 0, 1)



In [12]:
# === 9. PGD attack ===
def pgd_attack(model, images, labels, epsilon=0.02, alpha=0.005, steps=10):
    ori_images = images.clone().detach()
    perturbed = images.clone().detach()
    for _ in range(steps):
        perturbed.requires_grad = True
        outputs = model(perturbed)
        loss = F.cross_entropy(outputs, labels)
        model.zero_grad()
        loss.backward()
        grad = perturbed.grad.detach()
        perturbed = perturbed + alpha * grad.sign()
        perturbed = torch.min(torch.max(perturbed, ori_images - epsilon), ori_images + epsilon)
        perturbed = torch.clamp(perturbed, 0, 1).detach()
    return perturbed


In [13]:
# === 10. Patch attack ===
def patch_attack(model, images, labels, epsilon=0.5, patch_size=32):
    attacked = images.clone().detach()
    for i in range(images.size(0)):
        x, y = np.random.randint(0, 224 - patch_size + 1, 2)
        patch = attacked[i:i+1, :, y:y+patch_size, x:x+patch_size]
        adv_patch = fgsm_attack(model, patch.clone(), labels[i:i+1], epsilon)
        attacked[i, :, y:y+patch_size, x:x+patch_size] = adv_patch
    return attacked


In [14]:
# === 11. Visualize original vs adversarial ===
def visualize(original, adversarial, filename):
    inv_normalize = transforms.Normalize(
        mean=(-mean_norms / std_norms).tolist(),
        std=(1 / std_norms).tolist()
    )
    orig = inv_normalize(original.detach().cpu()).permute(1, 2, 0).numpy()
    adv = inv_normalize(adversarial.detach().cpu()).permute(1, 2, 0).numpy()
    fig, ax = plt.subplots(1, 2)
    ax[0].imshow(np.clip(orig, 0, 1))
    ax[0].set_title("Original")
    ax[1].imshow(np.clip(adv, 0, 1))
    ax[1].set_title("Adversarial")
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()


In [15]:
# === 12. Run full pipeline ===
def run_all():
    print("=== Task 1: Original Accuracy ===")
    top1, top5 = evaluate(model, testloader)
    print(f"Top-1: {top1:.4f}, Top-5: {top5:.4f}")

    print("=== Task 2: FGSM Attack ===")
    fgsm_images, labels = [], []
    for images, targets in tqdm(testloader):
        images, targets = images.to(device), targets.to(device)
        adv = fgsm_attack(model, images, targets, epsilon=0.02)
        fgsm_images.append(adv.cpu())
        labels.append(targets.cpu())
        visualize(images[0], adv[0], "fgsm_example.png")
    adv_dataset1 = torch.utils.data.TensorDataset(torch.cat(fgsm_images), torch.cat(labels))
    adv_loader1 = DataLoader(adv_dataset1, batch_size=16)
    top1_fgsm, top5_fgsm = evaluate(model, adv_loader1)
    print(f"FGSM Top-1: {top1_fgsm:.4f}, Top-5: {top5_fgsm:.4f}")

    print("=== Task 3: PGD Attack ===")
    pgd_images = []
    for images, targets in tqdm(testloader):
        images, targets = images.to(device), targets.to(device)
        adv = pgd_attack(model, images, targets)
        pgd_images.append(adv.cpu())
        visualize(images[0], adv[0], "pgd_example.png")
    adv_dataset2 = torch.utils.data.TensorDataset(torch.cat(pgd_images), torch.cat(labels))
    adv_loader2 = DataLoader(adv_dataset2, batch_size=16)
    top1_pgd, top5_pgd = evaluate(model, adv_loader2)
    print(f"PGD Top-1: {top1_pgd:.4f}, Top-5: {top5_pgd:.4f}")

    print("=== Task 4: Patch Attack ===")
    patch_images = []
    for images, targets in tqdm(testloader):
        images, targets = images.to(device), targets.to(device)
        adv = patch_attack(model, images, targets, epsilon=0.5)
        patch_images.append(adv.cpu())
        visualize(images[0], adv[0], "patch_example.png")
    adv_dataset3 = torch.utils.data.TensorDataset(torch.cat(patch_images), torch.cat(labels))
    adv_loader3 = DataLoader(adv_dataset3, batch_size=16)
    top1_patch, top5_patch = evaluate(model, adv_loader3)
    print(f"Patch Top-1: {top1_patch:.4f}, Top-5: {top5_patch:.4f}")

    print("=== Task 5: Transferability to DenseNet-121 ===")
    densenet = models.densenet121(weights='IMAGENET1K_V1').to(device).eval()
    accs = {
        "Original": evaluate(densenet, testloader),
        "FGSM": evaluate(densenet, adv_loader1),
        "PGD": evaluate(densenet, adv_loader2),
        "Patch": evaluate(densenet, adv_loader3)
    }
    for name, (t1, t5) in accs.items():
        print(f"{name}: Top-1 = {t1:.4f}, Top-5 = {t5:.4f}")


In [16]:
if __name__ == "__main__":
    run_all()


=== Task 1: Original Accuracy ===


100%|██████████| 32/32 [00:13<00:00,  2.43it/s]


Top-1: 0.7600, Top-5: 0.9420
=== Task 2: FGSM Attack ===


100%|██████████| 32/32 [00:00<00:00, 40.26it/s]


FGSM Top-1: 0.2640, Top-5: 0.5020
=== Task 3: PGD Attack ===


...

PGD Top-1: 0.0040, Top-5: 0.0660
=== Task 4: Patch Attack ===


100%|██████████| 32/32 [00:01<00:00, 29.20it/s]


Patch Top-1: 0.7520, Top-5: 0.9380
=== Task 5: Transferability to DenseNet-121 ===


100%|██████████| 32/32 [00:02<00:00, 12.32it/s]

Original: Top-1 = 0.7460, Top-5 = 0.9360
FGSM: Top-1 = 0.4260, Top-5 = 0.6620
PGD: Top-1 = 0.3980, Top-5 = 0.6280
Patch: Top-1 = 0.7380, Top-5 = 0.9220
